In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [11]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-02-20 03:00:00+00:00


In [12]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [13]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-02-19 21:21:42,921 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:21:42,936 INFO: Initializing external client
2025-02-19 21:21:42,937 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:21:43,988 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [14]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [15]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,227,0.0,2025-02-20 02:00:00+00:00
1,231,42.0,2025-02-20 02:00:00+00:00
2,243,0.0,2025-02-20 02:00:00+00:00
3,203,0.0,2025-02-20 02:00:00+00:00
4,143,82.0,2025-02-20 02:00:00+00:00
...,...,...,...
501,160,0.0,2025-02-20 03:00:00+00:00
502,11,0.0,2025-02-20 03:00:00+00:00
503,45,4.0,2025-02-20 03:00:00+00:00
504,146,1.0,2025-02-20 03:00:00+00:00


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  506 non-null    int32                  
 1   predicted_demand    506 non-null    float64                
 2   pickup_hour         506 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.0 KB


In [17]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-02-19 21:21:46,490 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:21:46,502 INFO: Initializing external client
2025-02-19 21:21:46,503 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:21:47,615 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Current UTC time: 2025-02-20 02:21:46.490266+00:00
Next hour: 2025-02-20 03:00:00+00:00
Querying for date range: 2025-02-20 to 2025-02-21
Filtering for hour: 2025-02-20 03:00
Found 253 records


In [18]:
now = datetime.now(timezone.utc)

In [19]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
253,259,0.0,2025-02-20 03:00:00+00:00
254,188,1.0,2025-02-20 03:00:00+00:00
255,20,0.0,2025-02-20 03:00:00+00:00
256,189,1.0,2025-02-20 03:00:00+00:00
257,2,0.0,2025-02-20 03:00:00+00:00
...,...,...,...
501,160,0.0,2025-02-20 03:00:00+00:00
502,11,0.0,2025-02-20 03:00:00+00:00
503,45,4.0,2025-02-20 03:00:00+00:00
504,146,1.0,2025-02-20 03:00:00+00:00


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  506 non-null    int32                  
 1   predicted_demand    506 non-null    float64                
 2   pickup_hour         506 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 10.0 KB


In [21]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [22]:
dt = current_date.ceil('h')


In [23]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-02-19 21:21:49,275 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:21:49,286 INFO: Initializing external client
2025-02-19 21:21:49,286 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:21:50,399 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [24]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 


,pickup_location_id,predicted_demand,pickup_hour


In [25]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,227,0.0,2025-02-20 02:00:00+00:00
1,231,42.0,2025-02-20 02:00:00+00:00
2,243,0.0,2025-02-20 02:00:00+00:00
3,203,0.0,2025-02-20 02:00:00+00:00
4,143,82.0,2025-02-20 02:00:00+00:00
...,...,...,...
501,160,0.0,2025-02-20 03:00:00+00:00
502,11,0.0,2025-02-20 03:00:00+00:00
503,45,4.0,2025-02-20 03:00:00+00:00
504,146,1.0,2025-02-20 03:00:00+00:00


In [26]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [27]:
predictions = fetch_next_hour_predictions()

2025-02-19 21:21:53,523 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:21:53,539 INFO: Initializing external client
2025-02-19 21:21:53,540 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:21:54,621 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Current UTC time: 2025-02-20 02:21:53.523702+00:00
Next hour: 2025-02-20 03:00:00+00:00
Found 253 records


In [28]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-02-19 21:21:55,503 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-19 21:21:55,510 INFO: Initializing external client
2025-02-19 21:21:55,510 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-19 21:21:56,582 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


In [29]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [30]:
df["pickup_hour"].max()

Timestamp('2025-02-20 03:00:00+0000', tz='Etc/UTC')

In [31]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-02-20 03:00:00+00:00


In [32]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
253,259,0.0,2025-02-20 03:00:00+00:00
254,188,1.0,2025-02-20 03:00:00+00:00
255,20,0.0,2025-02-20 03:00:00+00:00
256,189,1.0,2025-02-20 03:00:00+00:00
257,2,0.0,2025-02-20 03:00:00+00:00
...,...,...,...
501,160,0.0,2025-02-20 03:00:00+00:00
502,11,0.0,2025-02-20 03:00:00+00:00
503,45,4.0,2025-02-20 03:00:00+00:00
504,146,1.0,2025-02-20 03:00:00+00:00
